In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import pandas as pd
import numpy as np

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [4]:
with open('/content/drive/MyDrive/프로젝트/data/KLyricSentenceDFFinal.p', 'rb') as f:
    df = pickle.load(f)
df

,title,artist,lyric,genre
0,내 손을 잡아,아이유,"[느낌이 오잖아, 떨리고 있잖아, 언제까지 눈치만 볼 거니, 네 맘을 말해봐, 딴청...",balad
1,너의 모든 순간 (Original),성시경,"[이윽고 내가 한눈에, 너를 알아봤을 때, 모든 건 분명 달라지고 있었어, 내 세상...",balad
2,Timeless,SG 워너비,"[어쩜 살아가다 보면, 한 번은 날 찾을지 몰라, 난 그 기대 하나로, 오늘도 힘겹...",balad
3,바람이 불었으면 좋겠어,길구봉구,"[웃는 모습이 너무 아름다운, 그런 너를 기억하면서, 괜찮아 넌 잘할거라 말하던, ...",balad
4,봄 사랑 벚꽃 말고,HIGH4 (하이포),"[길었던 겨우내 줄곧 품이 좀 남는, 밤색 코트, 그 속에 나를 쏙 감추고 걸음을,...",balad
...,...,...,...,...
3391,소녀,뮤뭉,"[손만 닿아도 떨어져버릴 것 같은, 꽃잎 같은 모습을 하고 있는 널 보았어, 아 소...",folk
3392,꿈처럼 이내 사라져 버리네,박연두,"[바람은 기억하고 있었고, 바다는 널 품에 안고 있었어, 우리는 늘 그렇듯 살며 잊...",folk
3393,과일가게총각,나날(NaaNaL),"[우리 동네 과일가게에서, 일하는 총각이 좋아, 갈색머리 우렁찬 목소리에, 나는 빠...",folk
3394,Attitude,이보람,"[빼곡히도 적어 놓았던, 알 수 없던 나의 생각들이, 점점 줄어드는 공간이, 마침표...",folk


In [5]:
with open('/content/drive/MyDrive/프로젝트/모델/tokenizer1.p', 'rb') as f:
    tokenizer = pickle.load(f)

In [6]:
m = load_model('/content/drive/MyDrive/프로젝트/모델/LSTM2.h5')

In [7]:
from nltk import ConditionalFreqDist, ConditionalProbDist, MLEProbDist
from nltk.util import ngrams

In [64]:
stopwords = ['가', '예', '이', '리가', '하', '오', '지', '도', '마치']

In [ ]:
x_data = []
for i in range(len(df)):
    words = ' '.join(df.iloc[i,2]).split()
    print(words)
    x_data.append([w for w in words if w not in stopwords])

In [66]:
data_l = []
for lyric in x_data:
    data = ngrams(lyric, 2, pad_left=True, pad_right = True, left_pad_symbol='SS', right_pad_symbol = 'SE')
    data_l += [l for l in data]

In [67]:
cfd = ConditionalFreqDist(data_l)

In [ ]:
sorted(tokenizer.word_counts.items(), key = lambda x: x[1], reverse = True)[0]

('내', 10808)

In [68]:
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [ ]:
#니다 네요 군요 어요 

In [16]:
# 단어 여러개 넣어서 문맥이 있어보이는 척해보기
def sentence_generation(current_word, n, cpd = cpd, model = m, tokenizer = tokenizer): # 현재 단어, 반복할 횟수, 모델, 토크나이저 
    words = current_word.split()
    words_len = len(words)
    sentence = ''

    for i in range(words_len):
        try:
            current_word = cpd[words[i]].generate()
        except:
            current_word = cpd['내'].generate() # 없는 단어라면 가장 많이 등장하는 '내'에서 generate
        if words[i][-2] not in '니다네요군요어요잖아':
            if i == 0:
                sentence = sentence + ' ' + words[i]
            else:
                sentence = sentence + words[i]
        
        # n번 반복
        for _ in range(n):
            encoded = tokenizer.texts_to_sequences([current_word])[0]
            encoded = pad_sequences([encoded], maxlen=10-1, padding='pre')

            # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
            result = model.predict(encoded, verbose=0)
            result = np.argmax(result, axis=1)

            for word, index in tokenizer.word_index.items(): 
                # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                if index == result:
                    break

            # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
            current_word = current_word + ' '  + word

            # 예측 단어를 문장에 저장    
            sentence = sentence + ' ' + word
        
        sentence += '\n'
        # sentence = init_word + ' ' + sentence

    return sentence[1:]

꼴보기 싫은 단어가 등장하진 않는듯?




In [17]:
word = input()
print(sentence_generation(word, 5))

집에 갈까 한다
집에 내 머리맡에 싶을 쉬네 어디에
갈까 날에 생각나 어딘지를 몰라 몰라
한다 내게 잡고 바닷가 기억들도 위



In [33]:
cfd['휴가']

FreqDist({'때마다': 2, '하루끝에서': 2, '나왔어': 1, '더': 1, '한': 1, '떠날': 1, '나갈게': 1, '떠났던': 1})

In [51]:
cpd['더위'].generate()

'마치'

MLED generate를 중간중간 해보기

In [69]:
# 단어 여러개 넣어서 문맥이 있어보이는 척해보기
def sentence_generation(current_word, n, cpd = cpd, model = m, tokenizer = tokenizer): # 현재 단어, 반복할 횟수, 모델, 토크나이저 
    words = current_word.split()
    words_len = len(words)
    sentence = ''

    for i in range(words_len):
        try:
            current_word = cpd[words[i]].generate()
        except:
            current_word = cpd['내'].generate() # 없는 단어라면 가장 많이 등장하는 '내'에서 generate
        if words[i][-2] not in '니다네요군요어요잖아싶어':
            if i == 0:
                sentence = sentence + ' ' + current_word
            else:
                sentence = sentence + ' ' + current_word
        
        # n번 반복
        for _ in range(n):
            encoded = tokenizer.texts_to_sequences([current_word])[0]
            encoded = pad_sequences([encoded], maxlen=10-1, padding='pre')

            # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
            result = model.predict(encoded, verbose=0)
            result = np.argmax(result, axis=1)

            for word, index in tokenizer.word_index.items(): 
                # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                if index == result:
                    break

            # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
            current_word = current_word + ' '  + word

            # 예측 단어를 문장에 저장    
            sentence = sentence + ' ' + word
        
        sentence += '\n'
        # sentence = init_word + ' ' + sentence

    return sentence[1:]

In [70]:
word = input()
print(sentence_generation(word, 5))

바다 더위 여름 휴가
밀려오는 파도 속에 그치고 나 편하게
 덥고 더워 더워 덥다 덥다 춰봐
 날씨 더우니까 성질 나는 마당에 한다네
 떠났던 듯 말해 줘 건 사랑이라면

